# Generate job files for MSI computing
Main feature is to get all subfolders in a directory, and set up the output folders correspondingly

In [1]:
import myImageLib as mil
import os
import numpy as np
from tool_utils import *

Generate a header for job file:
```Python
def header(walltime, mem, code_path, to_file=None):
    """
    walltime -- estimate running time (hr)
    mem -- memory needed (gb)
    code_path -- the folder in which the code can be found
    to_file -- file object to write to. If you want to print in console, set to None (default)
```

In [3]:
# test header()
header(8, 3, '~/code/Python/Correlation/src/py_files', nodes=2)

#!/bin/bash -l
#PBS -l walltime=8:00:00,nodes=2:ppn=8,mem=3gb
#PBS -m abe
#PBS -M liux3141@umn.edu

cd ~/code/Python/Correlation/src/py_files
module load python3
source activate pythonEnv


In [4]:
# log
print(data_log())

{'08032020': {'num': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'fps': [30, 30, 30, 30, 30, 30, 30, 10, 10, 10, 10, 10, 10, 10, 10]}, '08042020': {'num': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'fps': [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]}, '08052020': {'num': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'fps': [30, 30, 30, 30, 30, 30, 30, 30, 30, 10, 10, 10]}, '08062020': {'num': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'fps': [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 10]}}


- 00.pbs: python to_tif nd2Dir
- 01.pbs: python piv_imseq.py img_folder output_folder winsize overlap fps smooth
- 02.pbs: python df2_kinetics.py img_folder kinetics_folder seg_length
- 03.pbs: python overall_intensity.py folder folder_out
- 04.pbs: python corr_imseq.py input_folder output_folder wsize step


- 05.pbs: python div2.py piv_folder imgDir output_folder winsize step
- 06.pbs: python cav_imseq.py piv_folder output_folder
- 07.pbs: python dc_adv.py folder_img folder_piv folder_out interval fps step
- 08.pbs: python div_x_dcadv.py folder_div folder_dcadv folder_out
- 09.pbs: python energy_order.py piv_folder eo_folder mpp fps
- 10.pbs: python piv_overlay.py pivDataFolder imgFolder output_folder sparcity
- 11.pbs: python piv_imseq.py img_folder output_folder winsize overlap fps smooth (no smoothing)
- 12.pbs: python flow-derivs.py piv_folder img_folder flow_folder winsize step
- 13.pbs: python local_df.py img_folder save_folder seg_length winsize step
- 14.pbs: python local_df.py img_folder save_folder seg_length winsize step (a range of seg_length)
- 15.pbs: python small_imseq.py img_folder out_folder wsize step
- 16.pbs: python concgrad_velocity.py small_img_folder piv_folder out_folder
- 17.pbs: python energy_spectrum.py piv_folder out_folder percentile

In [10]:
# Global dirs
################# INPUT ##############################
folder = r'E:\Github\Python\job_files\08042020' # job file folder
date = '08032020' # data date
######################################################
if os.path.exists(folder) == False:
    os.makedirs(folder)
data_path = '~/Data/' + date
num = data_log()[date]['num']
fpsL = data_log()[date]['fps']

nd2Dir = data_path + '/{0:02d}.nd2'
img_folder = data_path + '/{0:02d}/8-bit'
piv_folder = data_path + '/piv_imseq/{0:02d}'
i_folder = data_path + '/overall_intensity/{0:02d}'
dxd_folder = data_path + '/div_x_dcadv/{0:02d}'
kinetics_folder = data_path + '/df2_kinetics/{0:02d}'
dcadv_folder = data_path + '/dc_adv/{0:02d}/dt={1:d}'
dcadv_master_folder = data_path + '/dc_adv/{0:02d}'
corr_folder = data_path + '/corr_imseq/{0:02d}'
cav_folder = data_path + '/cav_imseq/{0:02d}'
div_folder = data_path + '/div2/{0:02d}'
eo_folder = data_path + '/energy_order/{0:02d}'
overlay_folder = data_path + '/piv_overlay/{0:02d}'
piv_nosmooth_folder = data_path + '/piv_nosmooth/{0:02d}'
flow_folder = data_path + '/flow_derivs/{0:02d}'
local_df_folder = data_path + '/local_df/{0:02d}'
small_img_folder = data_path + '/small_imseq/{0:02d}'
concgrad_velocity_folder = data_path + '/concgrad_velocity/{0:02d}'
es_folder = data_path + '/energy_spectrum/{0:02d}'

In [11]:
script = 'to_tif.py'
job_num = 0 # job file number
walltime = 3 # hours
mem = 3 # gb
code_dir = '~/code/Python/generic_proc/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(('python ' + script + ' ' + nd2Dir +' &\n').format(n))
    f.write('wait')

In [12]:
script = 'piv_imseq.py'
job_num = 1 # job file number
walltime = 20 # hours
mem = 3 # gb
code_dir = '~/code/Python/PIV/src/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(('python ' + script + ' ' + img_folder + ' ' + piv_folder + ' 50 25 {1:d} 1' +' &\n').format(n, fps))
    f.write('wait')

In [13]:
script = 'df2_kinetics.py'
job_num = 2 # job file number
walltime = 10 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + img_folder + ' ' + kinetics_folder + ' 50' +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [14]:
# python overall_intensity.py folder folder_out
script = 'overall_intensity.py'
job_num = 3 # job file number
walltime = 3 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + img_folder + ' ' + i_folder +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [15]:
# python corr_imseq.py img_folder corr_folder wsize step
script = 'corr_imseq.py'
job_num = 4 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + img_folder + ' ' + corr_folder + ' 25 25' +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [16]:
# python div2.py piv_folder imgDir output_folder winsize step
script = 'div2.py'
job_num = 5 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + piv_folder + ' ' + img_folder + ' ' + div_folder + ' 50 25' +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [17]:
# python cav_imseq.py piv_folder output_folder
script = 'cav_imseq.py'
job_num = 6 # job file number
walltime = 5 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + piv_folder + ' ' + cav_folder +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n))
    f.write('wait')

In [17]:
# python dc_adv.py folder_img folder_piv folder_out interval fps step
script = 'dc_adv.py'
job_num = 7 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
dtL = range(0, 15)
cmd = ('python ' + script + ' ' + img_folder + ' ' + piv_folder + ' ' + dcadv_folder + ' {1:d} {2:d} 25' +' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)   
    for n, fps in zip(num, fpsL):
        for dt in dtL:
            f.write(cmd.format(n, dt*fps, fps))
    f.write('wait')

In [10]:
# python div_x_dcadv.py folder_div folder_dcadv folder_out
script = 'div_x_dcadv.py'
job_num = 8 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + div_folder + ' ' + dcadv_master_folder + ' ' + dxd_folder + ' &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n, 2*fps, fps))
    f.write('wait')

In [25]:
# python energy_order.py piv_folder eo_folder mpp fps
script = 'energy_order.py'
job_num = 9 # job file number
walltime = 8 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + piv_folder + ' ' + eo_folder + ' 0.33 {2:d} &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n, 2*fps, fps))
    f.write('wait')

In [7]:
# python piv_overlay.py pivDataFolder imgFolder output_folder sparcity
script = 'piv_overlay.py'
job_num = 10 # job file number
walltime = 16 # hours
mem = 3 # gb
code_dir = '~/code/Python/PIV/src/py_files' # python script dir
cmd = ('python ' + script + ' ' + piv_folder + ' ' + img_folder + ' ' + overlay_folder + ' 2 &\n')
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(cmd.format(n, 2*fps, fps))
    f.write('wait')

In [6]:
script = 'piv_imseq.py'
job_num = 11 # job file number
walltime = 20 # hours
mem = 3 # gb
code_dir = '~/code/Python/PIV/src/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(('python ' + script + ' ' + img_folder + ' ' + piv_folder + ' 50 25 {1:d} 0' +' &\n').format(n, fps))
    f.write('wait')

In [15]:
script = 'flow_derivs.py'
job_num = 12 # job file number
walltime = 8 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(('python ' + script + ' ' + piv_folder + ' ' + img_folder + ' ' + flow_folder + ' 50 25 ' + ' &\n').format(n, fps))
    f.write('wait')

In [16]:
script = 'local_df.py'
job_num = 13 # job file number
walltime = 8 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):       
        f.write(('python ' + script + ' ' + img_folder + ' ' + local_df_folder + ' 50 50 25' +' &\n').format(n, fps))
    f.write('wait')

In [13]:
script = 'local_df.py'
job_num = 14 # job file number
walltime = 8 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):
        for seg_length in range(10, 100, 10):
            f.write(('python ' + script + ' ' + img_folder + ' ' + local_df_folder + '/dt={2:d} {2:d} 50 25' +' &\n').format(n, fps, seg_length))
    f.write('wait')

In [19]:
script = 'small_imseq.py'
job_num = 15 # job file number
walltime = 2 # hours
mem = 3 # gb
code_dir = '~/code/Python/generic_proc/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):
        f.write(('python ' + script + ' ' + img_folder + ' ' + small_img_folder + ' 50 25 &\n').format(n, fps))
    f.write('wait')

In [22]:
script = 'concgrad_velocity.py'
job_num = 16 # job file number
walltime = 4 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):
        f.write(('python ' + script + ' ' + small_img_folder + ' ' + piv_folder + ' ' + concgrad_velocity_folder + ' &\n').format(n, fps))
    f.write('wait')

In [11]:
script = 'energy_spectrum.py'
job_num = 17 # job file number
walltime = 4 # hours
mem = 3 # gb
code_dir = '~/code/Python/Correlation/src/py_files' # python script dir
with open(os.path.join(folder, '{:02d}.pbs'.format(job_num)), 'w') as f:
    header(walltime, mem, code_dir, to_file=f)    
    for n, fps in zip(num, fpsL):
        f.write(('python ' + script + ' ' +  piv_folder + ' ' + es_folder + ' 0.8 &\n').format(n, fps))
    f.write('wait')